# Exporting model from PyTorch to ONNX

In this tutorial, we describe how to use ONNX to convert a model defined
in PyTorch into the ONNX format.

ONNX exporter is part of the [PyTorch repository](http://pytorch.org/docs/master/onnx.html).

For working with this tutorial, you will need to install [onnx](https://github.com/onnx/onnx). You can get binary builds of onnx with
``conda install -c conda-forge onnx``.

``NOTE``: ONNX is under active development so for the best support consider building PyTorch master branch which can be installed by following
[the instructions here](https://github.com/pytorch/pytorch#from-source)

## Invoking exporter

Pretty much it's a matter of replacing `my_model(input)` with `torch.onnx.export(my_model, input, "my_model.onnx")` in your script.

### Limitations

The ONNX exporter is a trace-based exporter, which means that it operates by executing your model once, and exporting the operators which were actually run during this run. This means that if your model is dynamic, e.g., changes behavior depending on input data, the export won’t be accurate.

Similarly, a trace is might be valid only for a specific input size (which is one reason why we require explicit inputs on tracing). Most of the operators export size-agnostic versions and should work on different batch sizes or input sizes. We recommend examining the model trace and making sure the traced operators look reasonable.

In [1]:
import torch.onnx
help(torch.onnx.export)

Help on function export in module torch.onnx:

export(model, args, f, export_params=True, verbose=False, training=False, input_names=None, output_names=None, aten=False, export_raw_ir=False, operator_export_type=None, opset_version=None, _retain_param_name=True, do_constant_folding=True, example_outputs=None, strip_doc_string=True, dynamic_axes=None, keep_initializers_as_inputs=None, custom_opsets=None, enable_onnx_checker=True, use_external_data_format=False)
    Export a model into ONNX format.  This exporter runs your model
    once in order to get a trace of its execution to be exported;
    at the moment, it supports a limited set of dynamic models (e.g., RNNs.)
    
    Arguments:
        model (torch.nn.Module): the model to be exported.
        args (tuple of arguments): the inputs to
            the model, e.g., such that ``model(*args)`` is a valid
            invocation of the model.  Any non-Tensor arguments will
            be hard-coded into the exported model; any Tensor

## Trying it out on AlexNet

If you already have your model built, it's just a few lines:

In [2]:
import torch.onnx
import torchvision

# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.
dummy_input = torch.randn(1, 3, 224, 224)
# Obtain your model, it can be also constructed in your script explicitly
model = torchvision.models.alexnet(pretrained=True)
# Invoke export
torch.onnx.export(model, dummy_input, "../models/alexnet.onnx")

**That's it!**

## Inspecting model

You can also use ONNX tooling to check the validity of the resulting model or inspect the details

In [3]:
import onnx

# Load the ONNX model
model = onnx.load("../models/alexnet.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %input.1[FLOAT, 1x3x224x224]
) initializers (
  %classifier.1.bias[FLOAT, 4096]
  %classifier.1.weight[FLOAT, 4096x9216]
  %classifier.4.bias[FLOAT, 4096]
  %classifier.4.weight[FLOAT, 4096x4096]
  %classifier.6.bias[FLOAT, 1000]
  %classifier.6.weight[FLOAT, 1000x4096]
  %features.0.bias[FLOAT, 64]
  %features.0.weight[FLOAT, 64x3x11x11]
  %features.10.bias[FLOAT, 256]
  %features.10.weight[FLOAT, 256x256x3x3]
  %features.3.bias[FLOAT, 192]
  %features.3.weight[FLOAT, 192x64x5x5]
  %features.6.bias[FLOAT, 384]
  %features.6.weight[FLOAT, 384x192x3x3]
  %features.8.bias[FLOAT, 256]
  %features.8.weight[FLOAT, 256x384x3x3]
) {
  %17 = Conv[dilations = [1, 1], group = 1, kernel_shape = [11, 11], pads = [2, 2, 2, 2], strides = [4, 4]](%input.1, %features.0.weight, %features.0.bias)
  %18 = Relu(%17)
  %19 = MaxPool[kernel_shape = [3, 3], pads = [0, 0, 0, 0], strides = [2, 2]](%18)
  %20 = Conv[dilations = [1, 1], group = 1, kernel_shape = [5, 5], pads = [2, 2, 2

Notice that all parameters are listed as graph's inputs but they also have stored values initialized in `model.graph.initializers`.

## What's next

Check [PyTorch documentation on onnx file](http://pytorch.org/docs/master/onnx.html)
Take a look at [other tutorials, including importing of ONNX models to other frameworks](https://github.com/onnx/tutorials/tree/master/tutorials)

In [4]:
!pip list

Package                Version
---------------------- -------------------
absl-py                0.9.0
adabound               0.0.5
appdirs                1.4.4
asn1crypto             0.24.0
attrs                  17.4.0
Automat                0.6.0
backcall               0.2.0
bleach                 3.1.5
blinker                1.4
cachetools             4.1.1
certifi                2018.1.18
cffi                   1.14.0
chardet                3.0.4
click                  6.7
cloud-init             19.4
cloudpickle            1.4.1
colorama               0.3.7
command-not-found      0.3
configobj              5.0.6
constantly             15.1.0
cryptography           2.1.4
cycler                 0.10.0
Cython                 0.29.17
decorator              4.4.2
defusedxml             0.6.0
distro-info            0.18ubuntu0.18.04.1
entrypoints            0.3
Flask                  1.1.2
future                 0.18.2
google-auth            1.18.0
google-auth-oauthlib   0.4.1
grpcio   